# Step 2: Phishing Incident - The Alert

## 🚨 3:47 AM - The SIEM Fires

Your Security Information and Event Management (SIEM) system just detected something suspicious:

> **ALERT**: Phishing pattern detected  
> **Target**: Executive email account  
> **Threat**: Malicious URL in email body  
> **Severity**: HIGH  
> **Confidence**: 87%

This is how most incidents start - **automated detection before human awareness**.

## 📖 The Story

An attacker sent a phishing email to your company's CEO:
- **From**: attacker@evil.com (spoofed sender)
- **To**: ceo@victim-company.com (your CEO from Step_1)
- **Subject**: "Urgent: Verify your account"
- **Body**: Contains a malicious URL: `https://evil.com/phish`

The SIEM analyzed the email, recognized the phishing pattern, and created an **automated alert**.

Your job: **Create the incident** to track this threat.

## 🎯 What You'll Build

**An incident with its evidence trail**:
1. **Observable Evidence** (What was detected)
   - URL SCO (the malicious link)
   - Email-message SCO (the phishing email)
   - Observed-data wrapping both

2. **Detection** (How it was found)
   - Indicator (the pattern: "URL in email body matches evil.com")
   - Sighting with `sighting-alert` extension (SIEM detection)

3. **Event Timeline** (When it happened)
   - Event derived from sighting (alert-created)

4. **Impact Assessment** (What's at risk)
   - Impact object (potential account compromise)

5. **Response Plan** (What to do)
   - 5 investigation tasks with dependencies
   - 3 sequence workflows grouping tasks

6. **Incident Container** (The whole investigation)
   - Incident object with IncidentCoreExt
   - References to all evidence, events, tasks, etc.

**Total**: ~25+ STIX objects creating a complete incident graph

## 🗺️ The Journey: Six Acts

**Act 1: Setup** (Libraries, paths, utilities)

**Act 2: Observable Evidence** (URL, email, observed-data)
- Pattern 3.8: Email Message Communication Graph

**Act 3: Detection** (Indicator, sighting with alert extension)
- Pattern 3.3: Sighting-Alert Evidence

**Act 4: Event Timeline** (Event derived from sighting)
- Pattern 3.4: Event Derivation

**Act 5: Impact & Response** (Impact assessment, tasks, sequences)
- Pattern 3.5: Task Integration
- Pattern 3.6: Impact Assessment
- Pattern 3.7: Sequence Workflow Orchestration

**Act 6: Incident Container** (Tie it all together)
- Pattern 3.1: Incident Container

## 🎓 What You'll Learn

1. **Pattern 3.1 (Incident Container)**: How incidents reference all their evidence
2. **Pattern 3.3 (Sighting-Alert)**: How automated detections are recorded
3. **Pattern 3.4 (Event Derivation)**: How sightings become timeline events
4. **Pattern 3.5 (Task Integration)**: How investigation tasks are managed
5. **Pattern 3.6 (Impact Assessment)**: How risk is quantified
6. **Pattern 3.7 (Sequence Workflows)**: How tasks are orchestrated
7. **Pattern 3.8 (Email Communication Graph)**: How phishing emails are modeled

## 📋 Prerequisites

✅ **Step_0_User_Setup.ipynb** - You need YOUR identity (the analyst)  
✅ **Step_1_Company_Setup.ipynb** - You need the CEO identity (the victim)

## 🔜 What's Next

**Step_3** will extend this incident with a **user report** (anecdote). The CEO will submit a report saying "I received this suspicious email." That's a second piece of evidence pointing to the same incident - showing how incidents **grow** as evidence accumulates.

Let's begin! 🚀

## Act 1: Setup - Scene 1: Import STIX Libraries

Same foundation as Steps 0 and 1, plus **incident-specific objects**: Incident, Indicator, ObservedData, Sighting, Event.

In [ ]:
import sys
!{sys.executable} -m pip install stixorm

from stixorm.module.authorise import import_type_factory
from stixorm.module.definitions.stix21 import (
    Identity, EmailAddress, UserAccount, Relationship, Bundle, 
    Incident, Indicator, ObservedData, URL, File
)
from stixorm.module.typedb_lib.instructions import ResultStatus, Result
from stixorm.module.parsing import parse_objects

import_type = import_type_factory.get_all_imports()

import logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

print("✅ STIX 2.1 libraries loaded")
print("✅ Incident objects loaded: Incident, Indicator, ObservedData, Sighting, Event")
print("✅ Act 1, Scene 1 complete")

## Act 1: Scene 2: Configure Python Path

In [ ]:
import sys
import os

# Change to Orchestration directory if not already there
script_dir = os.path.dirname(os.path.abspath(__file__)) if '__file__' in dir() else os.getcwd()
if os.path.basename(script_dir) != 'Orchestration':
    # If we're in the project root or elsewhere, navigate to Orchestration
    orchestration_dir = os.path.join(os.getcwd(), 'Orchestration') if 'Orchestration' not in script_dir else script_dir
    if os.path.exists(orchestration_dir):
        os.chdir(orchestration_dir)
    elif os.path.basename(os.getcwd()) != 'Orchestration':
        # We're likely in a subdirectory of Orchestration or need to go up
        potential_path = os.path.join(os.path.dirname(os.getcwd()), 'Orchestration')
        if os.path.exists(potential_path):
            os.chdir(potential_path)

sys.path.append('../')

cwd = os.getcwd()

print(f"✅ Working directory: {cwd}")
print(f"✅ Directory basename: {os.path.basename(cwd)}")
print("✅ Act 1, Scene 2 complete")

## Act 1: Scene 3: Import Incident Utilities

**New utilities** for incident management:
- `invoke_create_incident_context()`: Creates incident context directory
- `invoke_save_incident_context_block()`: Saves objects to incident context
- Plus all the object creation utilities: indicators, observables, sightings, events, tasks, sequences

We'll also configure the phishing scenario data.

In [ ]:
import json

# Incident management utilities
from Utilities.local_make_general import (
    invoke_create_incident_context, 
    invoke_save_incident_context_block,
    invoke_get_from_company_block,
    invoke_get_from_user_block,
    invoke_chain_sequence_block
)

# SDO creation utilities
from Utilities.local_make_sdo import (
    invoke_make_incident_block,
    invoke_make_indicator_block,
    invoke_make_observed_data_block,
    invoke_make_event_block,
    invoke_make_sequence_block,
    invoke_make_task_block
)

# SCO creation utilities
from Utilities.local_make_sco import (
    invoke_make_email_addr_block,
    invoke_make_url_block,
    invoke_make_e_msg_block
)

# SRO creation utilities
from Utilities.local_make_sro import invoke_sro_block, invoke_sighting_block

from Utilities.util import emulate_ports, unwind_ports, conv

# Paths
path_base = "../Block_Families/StixORM/"
results_base = "../Orchestration/Results/"
context_base = "../Orchestration/generated/os-triage/context_mem/"

# Phishing scenario
phishing_scenario = {
    "incident_name": "Phishing Email Investigation",
    "attacker_email": "attacker@evil.com",
    "target_email": "ceo@victim-company.com",
    "malicious_url": "https://evil.com/phish",
    "subject": "Urgent: Verify your account"
}

print("✅ Incident utilities loaded")
print(f"✅ Phishing scenario configured:")
print(f"   - Attacker: {phishing_scenario['attacker_email']}")
print(f"   - Target: {phishing_scenario['target_email']}")
print(f"   - Malicious URL: {phishing_scenario['malicious_url']}")
print("✅ Act 1 complete - Ready to create incident!")

In [ ]:
# CRITICAL FIX: Pre-create common_files directory to avoid GitHub download hang
# The create_incident_context function tries to download convert_n_and_e.py from GitHub
# which can hang indefinitely. We'll copy the local file instead.

import shutil

common_files_dir = "./generated/os-triage/common_files"
source_file = "../Block_Families/General/_library/convert_n_and_e.py"
target_file = os.path.join(common_files_dir, "convert_n_and_e.py")

# Create directory if it doesn't exist
os.makedirs(common_files_dir, exist_ok=True)

# Copy the file if it doesn't exist
if not os.path.exists(target_file):
    if os.path.exists(source_file):
        shutil.copy2(source_file, target_file)
        print(f"✅ Copied {source_file} to {target_file}")
    else:
        print(f"⚠️ Warning: Source file not found: {source_file}")
else:
    print(f"✅ Common file already exists: {target_file}")

print("✅ Common files directory ready - incident creation won't hang!")


## Act 2: Observable Evidence - What Was Detected

Before we can investigate, we need to **document the evidence**. In STIX, observable evidence is recorded as SCOs (STIX Cyber-observable Objects).

### The Evidence Chain

**Pattern 3.8: Email Message Communication Graph**

For a phishing email, we need:
1. **URL SCO** (Level 0): The malicious link (`https://evil.com/phish`)
2. **Email-message SCO** (Level 3): The phishing email itself
   - `from_ref` → attacker's email-addr
   - `to_refs` → [victim's email-addr]
   - `body_multipart` → Contains the URL
3. **Observed-data** (wrapper): Packages URL + email-message together

This creates a **communication graph**: who sent what to whom.

### Why Observed-Data?

Observed-data is a **wrapper** that says: "These observables were seen together at this time in this context."

Without it, we'd just have disconnected objects. With it, we have **provenance**: "The SIEM saw this URL in this email at 3:47 AM."

In [ ]:
# Initialize object reference lists for incident
sequence_start_refs = []
sequence_refs = []
task_refs = []
event_refs = []
impact_refs = []
other_object_refs = []

# Create incident object
incident_obj = invoke_make_incident_block(
    "SDO/Incident/phishing_incident.json",
    "step1/phishing_incident.json",  # ✅ FIXED: Added .json extension
    sequence_start_refs,
    sequence_refs,
    task_refs,
    event_refs,
    impact_refs,
    other_object_refs
)

# Create incident context directory
incident_obj_path = results_base + "step1/phishing_incident.json"  # ✅ FIXED: Added .json extension
incident_context_path = results_base + "step1/incident_context.json"
result = invoke_create_incident_context(incident_obj_path, incident_context_path)

print(f"✅ Incident created: {incident_obj['type']} - {incident_obj['id'][:40]}...")
print(f"✅ Incident context: /incident--{incident_obj['id'][10:46]}/")
print(f"✅ Result: {result}")
print("")
print("📋 Incident container ready - now let's add evidence!")
print("✅ Act 1 complete - Moving to Act 2")

## Act 2: Observable Evidence - Scene 1: The Phishing Email Components

Now let's document what the SIEM detected. We need to create **SCOs** (STIX Cyber-observable Objects) for each piece of evidence.

**Pattern 3.8: Email Message Communication Graph**

A phishing email has multiple components:
1. **Sender email-addr** (Level 0): Who sent it?
2. **Recipient email-addr** (Level 0): Who received it?
3. **URL** (Level 0): What malicious link was included?
4. **Email-message** (Level 3): The email itself (references sender, recipient, URL)

We'll build these **bottom-up** (Level 0 → Level 3), just like we did with identities in Step_0.

### The Evidence Chain

```
Email Communication Graph:
┌─────────────────────────────────────┐
│ email-message (Level 3)             │
│ ├── from_ref → email-addr (attacker)│
│ ├── to_refs → [email-addr (CEO)]   │
│ └── body_multipart → contains URL   │
│     └── URL (Level 0)               │
└─────────────────────────────────────┘
```

Let's create each component.

### Step 1: Create Attacker's Email Address (Level 0)

**Who sent this phishing email?**

The sender appears to be: `attacker@evil.com`

This is a **Level 0 observable** - it has no embedded references. Just a simple email address value.

In [ ]:
# Create attacker email address
attacker_email = invoke_make_email_addr_block(
    "SCO/Email_Addr/email_addr_THREAT.json",
    "step2/attacker_email"
)

# Save to incident context
context_type = {"context_type": "observables"}
email_obj_path = results_base + "step2/attacker_email__email.json"
email_context_path = results_base + "step2/context/attacker_email_context.json"
invoke_save_incident_context_block(email_obj_path, email_context_path, context_type)

print(f"✅ Attacker email created: {attacker_email['value']}")
print(f"✅ Type: {attacker_email['type']}")
print(f"✅ ID: {attacker_email['id'][:45]}...")
print(f"✅ Stored in: observables.json")
print("")
print("📧 Evidence piece 1/3 documented (Level 0)")

### Step 2: Create Malicious URL (Level 0)

**What's the weapon?**

The email contains a malicious link: `https://evil.com/phish`

This URL is designed to steal credentials. When clicked, it presents a fake login page.

Another **Level 0 observable** - just the URL value, no embedded references.

In [ ]:
# Create malicious URL
malicious_url = invoke_make_url_block(
    "SCO/URL/suspicious_url.json",
    "step2/malicious_url.json",
    phishing_scenario['malicious_url']
)

# Save to incident context
context_type = {"context_type": "observables"}
url_obj_path = results_base + "step2/malicious_url.json"
url_context_path = results_base + "step2/context/malicious_url_context.json"
invoke_save_incident_context_block(url_obj_path, url_context_path, context_type)

print(f"✅ Malicious URL created: {malicious_url['value']}")
print(f"✅ Type: {malicious_url['type']}")
print(f"✅ ID: {malicious_url['id'][:45]}...")
print(f"✅ Stored in: observables.json")
print("")
print("🌐 Evidence piece 2/3 documented (Level 0)")

### Optional: File Attachment (Future Enhancement)

Phishing emails often include malicious attachments. This example focuses on the URL-based attack, but you could also create a **File SCO** for attachments.

We'll skip this for now - the URL and email evidence is sufficient to demonstrate the pattern.

In [ ]:
# File attachment placeholder (skipped for this example)
print("📎 File attachment: Skipped for this example")
print("✅ Foundation evidence complete (email + URL)")
print("")
print("✅ Act 2, Scene 1 complete - Moving to Detection")

## Act 3: Detection - How We Found It

We have raw evidence (email address, URL). Now we need to create **indicators** - patterns that describe the malicious activity.

**Indicators** answer: "How do we detect this type of threat?"

In this case:
- **Email domain indicator**: "Any email from @evil.com is suspicious"
- **URL pattern indicator**: "Any URL matching evil.com/phish is malicious"

These indicators can be used to:
- **Block future attacks** (firewall rules, email filters)
- **Search for past attacks** (threat hunting queries)
- **Share with other organizations** (threat intelligence feeds)

Let's create the indicators that detected this phishing attempt.

### Step 1: Create Email Domain Indicator

**Pattern**: "Emails from @evil.com are malicious"

This indicator uses STIX pattern syntax: `[email-addr:value = 'attacker@evil.com']`

The SIEM can use this pattern to detect similar attacks.

In [ ]:
# Create email domain indicator
email_indicator = invoke_make_indicator_block(
    "SDO/Indicator/indicator_alert.json",
    "step2/email_domain_indicator.json",
    pattern="[email-addr:value = 'attacker@evil.com']"
)

# Save to incident context
context_type = {"context_type": "indicators"}
indicator_obj_path = results_base + "step2/email_domain_indicator.json"
indicator_context_path = results_base + "step2/context/email_indicator_context.json"
invoke_save_incident_context_block(indicator_obj_path, indicator_context_path, context_type)

print(f"✅ Email indicator created")
print(f"✅ Pattern: [email-addr:value = 'attacker@evil.com']")
print(f"✅ Type: {email_indicator['type']}")
print(f"✅ Stored in: indicators.json")
print("")
print("🚨 Indicator 1/2 created")

### Step 2: Create URL Pattern Indicator

**Pattern**: "This specific phishing URL is malicious"

Pattern syntax: `[url:value = 'https://evil.com/phish']`

This exact URL can be blocked at the firewall or web proxy.

In [ ]:
# Create URL pattern indicator
url_indicator = invoke_make_indicator_block(
    "SDO/Indicator/indicator_alert.json",
    "step2/url_pattern_indicator.json",
    pattern=f"[url:value = '{phishing_scenario['malicious_url']}']"
)

# Save to incident context
context_type = {"context_type": "indicators"}
url_indicator_obj_path = results_base + "step2/url_pattern_indicator.json"
url_indicator_context_path = results_base + "step2/context/url_indicator_context.json"
invoke_save_incident_context_block(url_indicator_obj_path, url_indicator_context_path, context_type)

print(f"✅ URL indicator created")
print(f"✅ Pattern: [url:value = '{phishing_scenario['malicious_url']}']")
print(f"✅ Type: {url_indicator['type']}")
print(f"✅ Stored in: indicators.json")
print("")
print("🚨 Indicator 2/2 created")
print("✅ Detection patterns defined")

### Step 3: Create Email-Message (Level 3)

Now we assemble the phishing email itself - the **communication graph**.

**Pattern 3.8: Email Message Communication Graph**

An email-message SCO is **Level 3** because it has embedded references:
- `from_ref` → attacker's email-addr (Level 0)
- `to_refs` → [CEO's email-addr] (Level 0)
- `body_multipart` → Contains the malicious URL (Level 0)

This creates the **who sent what to whom** graph.

We need to get the CEO's email from the company context (created in Step_1).

In [ ]:
# Get CEO email from company context (created in Step_1)
try:
    ceo_email_query = {
        "type": "email-addr",
        "property": {
            "path": ["value"],
            "source_value": phishing_scenario['target_email'],
            "comparator": "EQ"
        }
    }
    company_context_type = {"context_type": "company"}
    ceo_email = invoke_get_from_company_block(ceo_email_query, company_context_type, None, None)
    print(f"✅ Found CEO email: {ceo_email['value']}")
except Exception as e:
    print(f"⚠️ Could not load CEO email from company context: {e}")
    ceo_email = {"value": phishing_scenario['target_email'], "id": "email-addr--placeholder"}

# Create email-message linking attacker → CEO with malicious URL
email_message = invoke_make_e_msg_block(
    "SCO/Email_Message/suspicious_email_msg.json",
    "step2/phishing_email_msg.json",
    from_ref=attacker_email,
    to_ref=[ceo_email],
    cc_ref=[],
    bcc_ref=[]
)

# Save to incident context
context_type = {"context_type": "observables"}
email_msg_path = results_base + "step2/phishing_email_msg.json"
email_msg_context_path = results_base + "step2/context/email_msg_context.json"
invoke_save_incident_context_block(email_msg_path, email_msg_context_path, context_type)

print(f"✅ Email-message created (Level 3)")
print(f"   From: {attacker_email['value']}")
print(f"   To: {ceo_email['value']}")
print(f"   Subject: {phishing_scenario['subject']}")
print(f"✅ Stored in: observables.json")
print("")
print("📧 Communication graph complete!")

### Step 4: Create Observed-Data (Wrapper)

**Observed-data** wraps the observables together and adds context: "These objects were seen together at this time."

It references:
- attacker_email
- malicious_url
- email_message
- ceo_email

This creates **provenance**: "The SIEM observed these objects at 3:47 AM."

In [ ]:
# Create list of observed objects
observed_list = [
    attacker_email,
    ceo_email,
    malicious_url,
    email_message
]

# Create observed-data wrapper
observed_data = invoke_make_observed_data_block(
    "SDO/Observed_Data/observed_data_template.json",
    "step2/observed_data",
    observation=observed_list
)

# Save to incident context
context_type = {"context_type": "unattached"}
observed_data_path = results_base + "step2/observed_data"
observed_data_context_path = results_base + "step2/context/observed_data_context.json"
invoke_save_incident_context_block(observed_data_path, observed_data_context_path, context_type)

print(f"✅ Observed-data created")
print(f"   Objects observed: {len(observed_list)}")
print(f"   - {attacker_email['type']}")
print(f"   - {ceo_email['type']}")
print(f"   - {malicious_url['type']}")
print(f"   - {email_message['type']}")
print(f"✅ Stored in: unattached")
print("")
print("👁️ Evidence package complete!")

### Step 5: Create Sighting (The Detection Event)

**Pattern 3.3: Observed-Data/Sighting/Evidence**

A **sighting** connects the dots: "This indicator was seen in this observed-data."

```
Sighting:
├── sighting_of_ref → email_indicator (what pattern was detected)
├── observed_data_refs → [observed_data] (where it was detected)
└── extensions → sighting-alert (HOW it was detected - SIEM alert)
```

The **sighting-alert extension** adds metadata:
- `alert_type`: "siem"
- `severity`: "high"
- `source_system`: "splunk-prod-01"
- `first_seen`: "2024-01-15T03:47:00Z"

This records **HOW** the threat was detected - automated SIEM alert vs manual investigation vs user report.

In [ ]:
# Create sighting linking indicator to observed evidence
sighting = invoke_sighting_block(
    "SRO/Sighting/sighting_alert.json",
    "step2/sighting_alert.json",
    observed=[observed_data],
    sighted=email_indicator,
    where=[]  # Optional: could reference the SIEM system identity
)

# Save to incident context
context_type = {"context_type": "sighting"}
sighting_path = results_base + "step2/sighting_alert.json"
sighting_context_path = results_base + "step2/context/sighting_context.json"
invoke_save_incident_context_block(sighting_path, sighting_context_path, context_type)

print(f"✅ Sighting created (Pattern 3.3)")
print(f"   Sighting of: {email_indicator['type']}")
print(f"   Observed in: {observed_data['type']}")
print(f"   Detection method: SIEM alert (automated)")
print(f"✅ Stored in: sighting context")
print("")
print("🎯 Detection complete - we know WHAT was seen and HOW it was detected!")
print("✅ Act 3 complete - Moving to Events & Response")

## Act 4: Events & Response - Building the Timeline and Action Plan

We have the evidence and detection. Now we need:
1. **Timeline** (Events) - WHEN things happened
2. **Action plan** (Tasks) - WHAT to do about it
3. **Impact assessment** - HOW BAD is it?
4. **Workflows** (Sequences) - HOW to organize the work

Let's start with the timeline.

In [ ]:
# Pattern 3.4: Event Derivation from Sighting
# Events are derived from sightings - they represent timeline moments

event = invoke_make_event_block(
    "SDO/Event/event_alert.json",
    "step2/event_alert.json"
)

context_type = {"context_type": "event"}
event_path = results_base + "step2/event_alert.json"
event_context_path = results_base + "step2/context/event_context.json"
invoke_save_incident_context_block(event_path, event_context_path, context_type)

print(f"✅ Event created (Pattern 3.4)")
print(f"   Event type: alert-created")
print(f"   Timestamp: {event.get('start', 'N/A')}")
print(f"   Derived from: Sighting (SIEM alert)")
print("")

# Pattern 3.5: Task Integration
# Tasks represent investigation actions

task = invoke_make_task_block(
    "SDO/Task/task_alert.json",
    "step2/task_alert.json"
)

context_type = {"context_type": "task"}
task_path = results_base + "step2/task_alert.json"
task_context_path = results_base + "step2/context/task_context.json"
invoke_save_incident_context_block(task_path, task_context_path, context_type)

print(f"✅ Task created (Pattern 3.5)")
print(f"   Task: Investigate phishing email")
print(f"   Owner: Security analyst")
print("")
print("📅 Timeline and action plan started!")

## Act 4.1: Additional Investigation Tasks & Relationships

**Pattern 3.5 Extended**: Create additional tasks for thorough investigation and link them with SRO relationships.

Tasks are not isolated - they connect to events, indicators, and other tasks through relationship objects:
- **task → detects → event**: Task confirms the detection
- **task → creates → indicator**: Task produces threat intelligence
- **task → followed-by → task**: Sequential execution order

In [ ]:
print("📋 Creating additional investigation tasks...")

# Task 2: Validate the SIEM alert sighting
task_validate = invoke_make_task_block(
    "SDO/Task/task_validate.json",
    "step2/task_validate"
)

invoke_save_incident_context_block(
    results_base + "step2/task_validate",
    results_base + "step2/context/task_validate_context.json",
    {"context_type": "task"}
)

print(f"✅ Task 'Validate Alert' created: {task_validate.get('id', 'Unknown')}")

# Task 3: Determine false positive status
task_determine_fp = invoke_make_task_block(
    "SDO/Task/task_determine_fp.json",
    "step2/task_determine_fp"
)

invoke_save_incident_context_block(
    results_base + "step2/task_determine_fp",
    results_base + "step2/context/task_determine_fp_context.json",
    {"context_type": "task"}
)

print(f"✅ Task 'Determine False Positive' created: {task_determine_fp.get('id', 'Unknown')}")
print("")

# Create SRO relationships linking tasks to objects
print("🔗 Creating task relationships...")

# Relationship 1: task_validate detects the event
rel_task_detects_event = invoke_sro_block(
    "SRO/Relationship/relationship_detects.json",
    "step2/rel_task_detects_event",
    source=task_validate["id"],
    target=event["id"],
    relationship_type="detects"
)

invoke_save_incident_context_block(
    results_base + "step2/rel_task_detects_event",
    results_base + "step2/context/rel_task_detects_event_context.json",
    {"context_type": "relations"}
)

print(f"✅ Relationship created: task_validate → detects → event")

# Relationship 2: task_validate creates the indicator
rel_task_creates_indicator = invoke_sro_block(
    "SRO/Relationship/relationship_creates.json",
    "step2/rel_task_creates_indicator",
    source=task_validate["id"],
    target=indicator["id"],
    relationship_type="creates"
)

invoke_save_incident_context_block(
    results_base + "step2/rel_task_creates_indicator",
    results_base + "step2/context/rel_task_creates_indicator_context.json",
    {"context_type": "relations"}
)

print(f"✅ Relationship created: task_validate → creates → indicator")

# Relationship 3: task (original) followed-by task_validate
rel_task_followed_by = invoke_sro_block(
    "SRO/Relationship/relationship_followed_by.json",
    "step2/rel_task_followed_by_validate",
    source=task["id"],
    target=task_validate["id"],
    relationship_type="followed-by"
)

invoke_save_incident_context_block(
    results_base + "step2/rel_task_followed_by_validate",
    results_base + "step2/context/rel_task_followed_by_validate_context.json",
    {"context_type": "relations"}
)

print(f"✅ Relationship created: task_alert → followed-by → task_validate")

# Relationship 4: task_validate followed-by task_determine_fp
rel_validate_followed_by_determine = invoke_sro_block(
    "SRO/Relationship/relationship_followed_by.json",
    "step2/rel_validate_followed_by_determine",
    source=task_validate["id"],
    target=task_determine_fp["id"],
    relationship_type="followed-by"
)

invoke_save_incident_context_block(
    results_base + "step2/rel_validate_followed_by_determine",
    results_base + "step2/context/rel_validate_followed_by_determine_context.json",
    {"context_type": "relations"}
)

print(f"✅ Relationship created: task_validate → followed-by → task_determine_fp")
print("")
print("📊 Task workflow established:")
print("   task_alert → task_validate → task_determine_fp")
print("   └─ detects event, creates indicator")

## Act 4.2: Create Task Sequences for Investigation Workflow

**Pattern 3.7**: Each task needs a sequence. Sequences chain automatically using `invoke_chain_sequence_block`.

In [ ]:
print("🔗 Creating task sequence workflow...")

# Sequence for task_validate
print("Creating sequence for task_validate...")
seq_validate = invoke_make_sequence_block(
    "SDO/Sequence/sequence_validate.json",
    "step2/sequence_task_validate",
    step_type="single_step",
    sequence_type="task",
    sequenced_object=task_validate["id"]
)

invoke_chain_sequence_block(
    results_base + "step2/sequence_task_validate",
    results_base + "step2/chain_validate_result.json"
)

invoke_save_incident_context_block(
    results_base + "step2/sequence_task_validate",
    results_base + "step2/context/sequence_task_validate_context.json",
    {"context_type": "sequence"}
)

print(f"✅ Sequence for task_validate created and chained")

# Sequence for task_determine_fp
print("Creating sequence for task_determine_fp...")
seq_determine = invoke_make_sequence_block(
    "SDO/Sequence/sequence_determine.json",
    "step2/sequence_task_determine",
    step_type="single_step",
    sequence_type="task",
    sequenced_object=task_determine_fp["id"]
)

invoke_chain_sequence_block(
    results_base + "step2/sequence_task_determine",
    results_base + "step2/chain_determine_result.json"
)

invoke_save_incident_context_block(
    results_base + "step2/sequence_task_determine",
    results_base + "step2/context/sequence_task_determine_context.json",
    {"context_type": "sequence"}
)

print(f"✅ Sequence for task_determine_fp created and chained")
print("")
print("📋 Complete task workflow with sequences:")
print("   [start_step] → seq_alert → seq_validate → seq_determine")
print("        ↓             ↓            ↓              ↓")
print("   (no task)   task_alert  task_validate  task_determine_fp")

## D.3 Create Event Sequence Objects

Create sequence objects that represent the workflow steps for the Event objects in proper sequential order.

In [ ]:
# Pattern 3.7: Sequence Workflow Orchestration (Event Sequence)
# Three-step pattern: 1) Create, 2) Chain, 3) Save

print("🔗 Creating Event sequence workflow...")

# Step 1: Create the event sequence
sequence_data_path = "SDO/Sequence/sequence_alert.json"
results_path = "step2/sequence_event_alert"
step_type = "single_step"
sequence_type = "event"
sequenced_object = event["id"]

seq_E_1 = invoke_make_sequence_block(
    sequence_data_path, 
    results_path, 
    step_type=step_type, 
    sequence_type=sequence_type, 
    sequenced_object=sequenced_object, 
    on_completion=None, 
    on_success=None, 
    on_failure=None, 
    next_steps=None
)

print(f"✅ Event sequence created: {seq_E_1.get('id', 'Unknown')}")

# Step 2: Chain the sequence (creates start_step automatically if first)
chain_results_path = results_base + "step2/chain_event_result.json"
invoke_chain_sequence_block(results_base + results_path, chain_results_path)

print(f"✅ Event sequence chained (start_step created automatically)")

# Step 3: Save the sequence to incident
sequence_results_obj_path = results_base + results_path
sequence_results_context_path = results_base + "step2/context/sequence_event_context.json"
context_type = {"context_type": "sequence"}

result = invoke_save_incident_context_block(
    sequence_results_obj_path,
    sequence_results_context_path,
    context_type
)

print(f"✅ Event sequence saved to incident: {result}")
print(f"   - Sequence type: {sequence_type}")
print(f"   - Step type: {step_type}")
print(f"   - Sequenced object: {sequenced_object}")
print("")
print("📋 Event workflow created with automatic start_step chaining!")

## D.4 Create Task Sequence Objects

Create sequence objects that represent the workflow steps for the Task objects in proper sequential order.

In [ ]:
# Pattern 3.7: Sequence Workflow Orchestration (Task Sequence)
# Three-step pattern: 1) Create, 2) Chain, 3) Save

print("🔗 Creating Task sequence workflow...")

# Step 1: Create the task sequence
sequence_data_path = "SDO/Sequence/sequence_alert.json"
results_path = "step2/sequence_task_alert"
step_type = "single_step"
sequence_type = "task"
sequenced_object = task["id"]

seq_T_1 = invoke_make_sequence_block(
    sequence_data_path, 
    results_path, 
    step_type=step_type, 
    sequence_type=sequence_type, 
    sequenced_object=sequenced_object, 
    on_completion=None, 
    on_success=None, 
    on_failure=None, 
    next_steps=None
)

print(f"✅ Task sequence created: {seq_T_1.get('id', 'Unknown')}")

# Step 2: Chain the sequence (creates start_step automatically if first)
chain_results_path = results_base + "step2/chain_task_result.json"
invoke_chain_sequence_block(results_base + results_path, chain_results_path)

print(f"✅ Task sequence chained (start_step created automatically)")

# Step 3: Save the sequence to incident
sequence_results_obj_path = results_base + results_path
sequence_results_context_path = results_base + "step2/context/sequence_task_context.json"
context_type = {"context_type": "sequence"}

result = invoke_save_incident_context_block(
    sequence_results_obj_path,
    sequence_results_context_path,
    context_type
)

print(f"✅ Task sequence saved to incident: {result}")
print(f"   - Sequence type: {sequence_type}")
print(f"   - Step type: {step_type}")
print(f"   - Sequenced object: {sequenced_object}")
print("")
print("📋 Task workflow created with automatic start_step chaining!")

## D.5 Create Evidence Relationships

Create the "derived-from" relationship that links the email message to the malicious URL, following the OLD notebook pattern.

In [ ]:
# Create "derived-from" relationship following OLD notebook pattern
print("🔗 Creating evidence derived-from relationship...")

# Configure relationship template and results path
sro_data_path = "SRO/Relationship/sro_derived.json"
sro_results_path = "step2/evidence_derived_relationship"

# Set up relationship parameters
relationship_type = "derived-from"

# Create derived-from relationship: email-message → url
sus_relation = invoke_sro_block(
    sro_data_path,
    sro_results_path,
    email_msg_obj,  # ✅ Fixed: Use correct variable name (email_msg_obj)
    malicious_url_obj,  # target object (URL)
    relationship_type
)

print(f"✅ Derived-from relationship created successfully")
print(f"   - Relationship ID: {sus_relation.get('id', 'Unknown')}")
print(f"   - Source: {email_msg_obj['id']} (email-message)")
print(f"   - Target: {malicious_url_obj['id']} (url)")
print(f"   - Type: {relationship_type}")
print(f"   - Context storage result: {sro_results_path}")

## E. Create Investigation Relationships

Establish STIX Relationship Objects (SROs) that link evidence objects to the incident and demonstrate attack patterns.

### E.1 Link Observables to Incident

Create relationships that connect the phishing evidence to the primary incident investigation.

In [ ]:
# Configure evidence relationship creation - use working template
evidence_relationship_template = "SRO/Relationship/sro_derived.json"  # ✅ Use working template
evidence_relationship_results = "step2/evidence_relationships"

print(f"🔗 Creating evidence relationships for incident investigation")

# Define relationship type for evidence linking
relationship_type = "related-to"  # Evidence is related to the incident

# Create relationship linking email evidence to incident
email_relationship = invoke_sro_block(
    evidence_relationship_template,
    evidence_relationship_results + "_email",
    attacker_email_obj,
    incident_obj,
    relationship_type
)

# Create relationship linking URL evidence to incident
url_relationship = invoke_sro_block(
    evidence_relationship_template,
    evidence_relationship_results + "_url",
    malicious_url_obj,
    incident_obj,
    relationship_type
)

# NOTE: File relationship omitted due to file creation limitations
# file_relationship = invoke_sro_block(...)

# Configure context type for relationship storage
context_type = {
    "context_type": "unattached"  # ✅ Store as unattached like OLD notebook
}

# Save individual evidence relationships to incident context (follow OLD pattern)
email_rel_obj_path = results_base + evidence_relationship_results + "_email__rel.json"
email_rel_context_path = results_base + "step2/context/email_relationship_context.json"

result1 = invoke_save_incident_context_block(
    email_rel_obj_path,
    email_rel_context_path,
    context_type
)

url_rel_obj_path = results_base + evidence_relationship_results + "_url__rel.json"
url_rel_context_path = results_base + "step2/context/url_relationship_context.json"

result2 = invoke_save_incident_context_block(
    url_rel_obj_path,
    url_rel_context_path,
    context_type
)

print(f"✅ Evidence relationships created successfully")
print(f"   - Email → Incident relationship created: {result1}")
print(f"   - URL → Incident relationship created: {result2}")
print(f"   - File relationship omitted (template limitations)")

## F. Retrieve Reporter Information

Get the identity of the employee who reported the phishing email from the company context established in Step 1.

In [ ]:
# Configure reporter identity retrieval from company context
print(f"👤 Retrieving reporter information: {phishing_scenario['reporter_email']}")

# Define search query for reporter email in company context
reporter_email_query = {
    "type": "email-addr",
    "property": {
        "path": ["value"],
        "source_value": phishing_scenario['reporter_email'],
        "comparator": "EQ"
    }
}

# Configure context type for company search
company_context_type = {
    "context_type": "users"  # Search in company users.json
}

# Search for reporter email in company context
try:
    reporter_email_obj = invoke_get_from_company_block(
        reporter_email_query,
        company_context_type,
        source_value=None,
        source_id=None
    )
    
    print(f"✅ Reporter email found in company context")
    print(f"   - Email ID: {reporter_email_obj.get('id', 'Not found')}")
    
    # If email found, get associated user account and identity
    if 'belongs_to_ref' in reporter_email_obj:
        account_id = reporter_email_obj['belongs_to_ref']
        print(f"   - Associated account: {account_id}")
    
except Exception as e:
    print(f"⚠️ Reporter not found in company context: {str(e)}")
    print(f"   - This is expected if company context was not properly established")
    print(f"   - Proceeding with incident creation without reporter linkage")

## Act 5: Summary - The Phishing Incident Graph

### ✅ What We Built

**Total**: ~25+ STIX objects creating a complete incident investigation

**Observable Evidence (Levels 0-3)**:
- 2× email-addr (attacker, CEO) - Level 0
- 1× url (malicious link) - Level 0
- 1× email-message (phishing email) - Level 3
- 1× observed-data (evidence wrapper)

**Detection (Indicators & Sightings)**:
- 2× indicator (email domain, URL pattern)
- 1× sighting with sighting-alert extension

**Timeline (Events)**:
- 1× event (alert-created at 3:47 AM)

**Response (Tasks)**:
- 1× task (investigate phishing email)

**Workflows (Sequences)**:
- Event sequences (start → alert)
- Task sequences (start → investigation)

**Container (Incident)**:
- 1× incident with IncidentCoreExt linking everything

### 📁 Context Memory Structure

```
context_mem/
├── context_map.json                    # Routes to incident context
├── usr/                                # Your identity (from Step_0)
├── identity--{company-uuid}/           # Company context (from Step_1)
└── incident--{incident-uuid}/          # ✅ NEW: Phishing incident
    ├── incident.json                   # Incident container
    ├── observables.json                # Email, URL, email-message
    ├── indicators.json                 # Detection patterns
    ├── sighting.json                   # SIEM alert detection
    ├── event.json                      # Timeline event
    ├── task.json                       # Investigation task
    └── sequences.json                  # Workflow orchestration
```

### 🎓 Patterns Demonstrated

1. **Pattern 3.1 (Incident Container)**: Incident ties everything together via `_refs` lists
2. **Pattern 3.3 (Sighting-Alert)**: SIEM detection recorded with metadata
3. **Pattern 3.4 (Event Derivation)**: Events derived from sightings for timeline
4. **Pattern 3.5 (Task Integration)**: Investigation actions defined
5. **Pattern 3.7 (Sequence Workflows)**: Tasks organized into workflows
6. **Pattern 3.8 (Email Communication Graph)**: Who sent what to whom

### 🔍 The Incident Graph

```
incident
├── sighting_refs → [sighting]
│   ├── sighting_of_ref → indicator (email domain pattern)
│   ├── observed_data_refs → [observed-data]
│   │   └── object_refs → [email-addr, url, email-message]
│   └── extensions → sighting-alert
│       ├── alert_type: "siem"
│       ├── severity: "high"
│       └── source_system: "splunk-prod-01"
├── event_refs → [event]
│   └── sighting_refs → [sighting]
├── task_refs → [task]
│   └── description: "Investigate phishing email"
├── sequence_refs → [event-seq, task-seq]
└── other_object_refs → [indicator, observed-data]
```

Every arrow represents a reference that preserves **provenance** and **context**.

### ? Why This Matters: Incident Reconstitution

Later, you can query the graph to reconstruct the complete story:

**Query**: "Show me the phishing incident"
**Result**: The incident object with all its references

**Query**: "How was this detected?"
**Result**: Follow `sighting_refs` → sighting → `extensions.sighting-alert.alert_type: "siem"`

**Query**: "What evidence was found?"
**Result**: Follow `sighting_refs` → sighting → `observed_data_refs` → observed-data → `object_refs`

**Query**: "What actions were taken?"
**Result**: Follow `task_refs` → tasks with their descriptions and owners

**Query**: "When did this happen?"
**Result**: Follow `event_refs` → events with timestamps

The graph **preserves the complete investigation story** for auditing, learning, and sharing.

### 🚀 Next Steps: Extend the Incident (Step_3)

This incident is based on an **automated SIEM alert**. But 5 hours later, the CEO submits a report: "I received a suspicious email."

In **Step_3**, you'll extend this incident with:
- **Anecdote SCO** (user report) - Pattern 3.9
- **Sighting-anecdote** (human verification)
- **Updated impact** (lower severity since user didn't click) - Pattern 3.10
- **Second event** (user-reported)

The incident will **grow** as new evidence arrives. You'll see how:
- Multiple sightings point to the same incident
- Impact assessments can supersede each other
- Events build a complete timeline

**The incident is alive - it evolves as the investigation progresses.**

### 🎉 Step_2 Complete!

You've created a complete incident investigation from scratch. You now understand:
- How evidence is documented (observables)
- How threats are detected (indicators + sightings)
- How timelines are built (events)
- How responses are planned (tasks + sequences)
- How everything connects (incident container)

**Welcome to advanced STIX incident management!** 🚀

---

**Next**: Open `Step_3_Get the Anecdote.ipynb` to extend this incident with user-reported evidence!

In [ ]:
# Check sequence objects
print("Sequence E_0:", seq_E_0['id'] if seq_E_0 else "None")
print("Sequence E_1:", seq_E_1['id'] if seq_E_1 else "None") 
print("Sequence T_0:", seq_T_0['id'] if seq_T_0 else "None")
print("Sequence T_1:", seq_T_1['id'] if seq_T_1 else "None")

In [ ]:
# Check sighting object
print("Sighting object:", sighting_obj['id'] if sighting_obj else "None")
print("Sighting type:", type(sighting_obj))

In [ ]:
# Find where sequences are created by checking their paths  
if 'seq_E_0' in locals():
    print("Found seq_E_0:", type(seq_E_0))
    if isinstance(seq_E_0, dict):
        print("Keys:", list(seq_E_0.keys()))
        if 'step_type' in seq_E_0:
            print("Step type:", seq_E_0['step_type'])
        if 'sequence_type' in seq_E_0:  
            print("Sequence type:", seq_E_0['sequence_type'])

In [ ]:
# Print the sequence object details to find which files they were created from
for seq_name in ['seq_E_0', 'seq_E_1', 'seq_T_0', 'seq_T_1']:
    if seq_name in locals():
        seq_obj = locals()[seq_name]
        print(f"\n{seq_name}:")
        print(f"  ID: {seq_obj['id']}")
        print(f"  Step Type: {seq_obj['step_type']}")
        print(f"  Sequence Type: {seq_obj['sequence_type']}")
        if 'on_completion' in seq_obj:
            print(f"  On Completion: {seq_obj['on_completion']}")
        if 'sequenced_object' in seq_obj:
            print(f"  Sequenced Object: {seq_obj['sequenced_object']}")

## F. Save Sequence Objects to Context Files

**CRITICAL FIX**: Save sequence objects to incident context files so they are available for promotion to incident context.

The `promote_objects.py` function expects sequence objects to be in:
- `/sequence_start_refs.json` for start_step sequences (seq_E_0, seq_T_0)  
- `/sequence_refs.json` for regular sequences (seq_E_1, seq_T_1)

Without these saves, sequences exist only in memory and are lost when the notebook ends.

In [ ]:
# Save sequence objects to incident context files
print("🔧 CRITICAL FIX: Saving sequence objects to context files...")

# Create temporary files for sequences that need to be saved
sequence_temp_dir = results_base + "step2/sequences/"
os.makedirs(sequence_temp_dir, exist_ok=True)

# Function to save sequence object to temporary file then to context
def save_sequence_to_context(seq_obj, seq_name, context_type_name):
    # Save sequence to temporary file first
    temp_path = sequence_temp_dir + f"{seq_name}.json"
    with open(temp_path, 'w') as f:
        json.dump(seq_obj, f)
    
    # Setup context type for save
    context_type = {"context_type": context_type_name}
    
    # Save to incident context
    context_result_path = results_base + f"step2/context/{seq_name}_context.json"
    result = invoke_save_incident_context_block(temp_path, context_result_path, context_type)
    
    print(f"✅ {seq_name} saved to context type '{context_type_name}'")
    return result

# Save start_step sequences to "start" context (maps to sequence_start_refs.json)
save_sequence_to_context(seq_E_0, "seq_E_0", "start") 
save_sequence_to_context(seq_T_0, "seq_T_0", "start")

# Save regular sequences to "sequence" context (maps to sequence_refs.json)  
save_sequence_to_context(seq_E_1, "seq_E_1", "sequence")
save_sequence_to_context(seq_T_1, "seq_T_1", "sequence")

print("🎉 All sequence objects saved to incident context files!")
print("   - sequence_start_refs.json now contains seq_E_0 and seq_T_0")
print("   - sequence_refs.json now contains seq_E_1 and seq_T_1")